In [1]:
import os
import json
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ["OPENAI_API_KEY"]
OPENROUTER_API_KEY = os.environ["OPENROUTER_API_KEY"]
os.environ["GROQ_API_KEY"]
os.environ["OPENROUTER_API_KEY"]
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_community.document_loaders import PyPDFLoader, TextLoader, Docx2txtLoader

In [2]:
def load_file(file_path: str):
    """
    Load content from a PDF, Word, or text file.
    """
    if file_path.endswith(".pdf"):
        loader = PyPDFLoader(file_path)
    elif file_path.endswith(".txt"):
        loader = TextLoader(file_path)
    else:
        raise ValueError("Unsupported file format. Please provide a .pdf, .docx, or .txt file.")
    documents = loader.load()
    return documents#"\n".join([doc.page_content for doc in documents])


# Paths to the job description and resume files
jd = "JD.txt"  # Replace with your file path
cv = "CV.pdf"  # Replace with your file path

def load_cv_jd(jd1= None, cv1= None):

    # Load the files
    try:
        job_description = load_file(jd1)
        resume1 = load_file(cv1)
        
        rjd = "\n".join([doc.page_content for doc in job_description])
        rcv = "\n".join([doc.page_content for doc in resume1])
        return rjd, rcv
        
    except Exception as e:
        print(f"An error occurred: {e}")
    
JD, CV = load_cv_jd(jd,cv)
print(CV)

Raviteja R
Senior Product Data Analyst, 
Certified Data Scientist
r2raviteja@gmail.com
 
+49 15216 442399
 
Munich, Germany
 
rraviteja
 
Profile  with 5 years of experience in 
analytics, A/B testing and KPI optimization. 
Proficient in building scalable dashboards 
and machine learning models to drive 
business impact. Available for immediate 
joining.
Education
Masters in Business Analytics & Data 
Science, EU Business School
March 2023 – March 2025    
Munich, Germany
GPA : 1,6/4 (German Scale,  1 = Highest)
Post Graduate Diploma in Data Science, 
IIIT Bangalore
April 2019 – September 2020
GPA - 3.2/4 (U.S Scale, with 4 = Highest)
Bachelor of Technology (ECE) | 
Specialization in Data Science, 
Lovely Professional University
June 2015 – April 2019
GPA : 7.76/10 (with 10 = Highest)
Projects
Thesis: AI-Driven Interview Bot
Developed an AI-powered interview bot 
leveraging Generative AI, Langchain, RAG 
and NLP, reducing early-stage interviewer 
workload by 60% and increasing candidat

In [3]:
from agno.agent import Agent
from agno.models.openai import OpenAIChat
from agno.tools.reasoning import ReasoningTools

data_extractor_agent = Agent(
    name="Data Extractor",
    role="Extract and structure personal information from unstructured text",
    model=OpenAIChat("gpt-4.1-nano"),
    tools=[ReasoningTools(add_instructions=True)],
    instructions=[
        "Extract specific fields from unstructured text input",
        "Identify and separate different data points",
        "Handle missing values (marked with '-') appropriately",
        "Return data in a structured dictionary format",
        "Verify extracted data for accuracy",
        "Handle both single-line and multi-line fields",
        "Preserve original formatting of URLs and special characters",
        "Convert empty fields to None or appropriate null representation"
    ],
    markdown=True,
)

In [4]:
query = """
    Extract the following fields from the resume below and return as JSON:
    - Email
    - password
    - First name
    - middle name (return None if '-')
    - Last name
    - Phone number
    - Language
    - Secondary Phone (return None if '-')
    - Street Address
    - City
    - State
    - Zip / Postal Code
    - Country
    - LinkedIn profile
    
    Example Output:
    {
        "Email": "abc@gmail.com",
        "password": "Asdfgh1234567@",
        "middle name": null
    }

    Resume to process:
    """+ CV
response = data_extractor_agent.run(query)
data_extractor_agent.print_response(response.content)

Output()

In [5]:
content_json = json.loads(response.content) 
content_json

{'Email': 'r2raviteja@gmail.com',
 'password': None,
 'First name': 'Raviteja',
 'middle name': None,
 'Last name': 'R',
 'Phone number': '+49 15216 442399',
 'Language': None,
 'Secondary Phone': None,
 'Street Address': None,
 'City': 'Munich',
 'State': None,
 'Zip / Postal Code': None,
 'Country': 'Germany',
 'LinkedIn profile': None}

In [6]:

content_json = json.loads(response.content) # {"key": "value"}  # Example content

# Specify the file path where you want to save the JSON
file_path = "user.txt"

# Write the JSON content to the text file
with open(file_path, 'w', encoding='utf-8') as file:
    json.dump(content_json, file, indent=4)  # indent for pretty formatting

print(f"JSON content has been written to {file_path}")

JSON content has been written to user.txt


In [7]:
import json

# Method 1: Basic loading
with open('user.txt', 'r', encoding='utf-8') as file:
    content_json = json.load(file)

print(content_json)

{'Email': 'r2raviteja@gmail.com', 'password': 'Asdf1234', 'First name': 'Raviteja', 'middle name': None, 'Last name': 'R', 'Phone number': '+49 15216 442399', 'Language': 'English', 'Secondary Phone': None, 'Street Address': None, 'City': 'Munich', 'State': None, 'Zip / Postal Code': None, 'Country': 'Germany', 'LinkedIn profile': None}


In [8]:
import json
from agno.memory.v2.memory import Memory
from agno.memory.v2.db.sqlite import SqliteMemoryDb
from agno.memory.v2.schema import UserMemory

def get_memory_db():
    """Create and return a memory database instance"""
    return SqliteMemoryDb(
        table_name="user_data",
        db_file="storage/user_memory.db"
    )

def get_user_value(user_id, field_name):
    """
    Retrieve a specific field value for a user
    Args:
        user_id (str): The user's ID (email)
        field_name (str): The field name to retrieve (e.g., "First name", "Phone number")
    Returns:
        str: The value if found, None otherwise
    """
    memory_db = get_memory_db()
    memory = Memory(db=memory_db)
    
    # Get all memories for the user
    user_memories = memory.get_user_memories(user_id=user_id)
    
    # Search for the specific field
    for m in user_memories:
        if m.memory.startswith(f"{field_name}:"):
            # Extract the value part after the colon and strip whitespace
            return m.memory.split(":", 1)[1].strip()
    
    return None

def get_all_user_values(user_id):
    """
    Retrieve all stored values for a user as a dictionary
    Args:
        user_id (str): The user's ID (email)
    Returns:
        dict: Dictionary of field names and their values
    """
    memory_db = get_memory_db()
    memory = Memory(db=memory_db)
    
    user_values = {}
    user_memories = memory.get_user_memories(user_id=user_id)
    
    for m in user_memories:
        # Split memory string into field name and value
        if ":" in m.memory:
            field, value = m.memory.split(":", 1)
            user_values[field.strip()] = value.strip()
    
    return user_values

def store_user_data(user_content):
    """
    Store user data from JSON content into the memory database
    Args:
        user_content (str): JSON string containing user data
    """
    try:
        # Parse JSON if it's a string
        if isinstance(user_content, str):
            user_data = json.loads(user_content)
        else:
            user_data = user_content

        # Create a SQLite memory database
        memory_db = get_memory_db()

        # Initialize Memory with the storage backend
        memory = Memory(db=memory_db)

        # Use email as user_id if available, otherwise use a default
        user_id = user_data.get("Email", user_data.get("email", "default_user"))

        # Store each piece of user information as a separate memory
        for key, value in user_data.items():
            if value is not None:  # Only store non-null values
                # Clean the key for topic (remove special characters, lowercase)
                topic = key.lower().replace(" ", "_").replace(",", "").replace(".", "")
                
                memory.add_user_memory(
                    memory=UserMemory(
                        memory=f"{key}: {value}",
                        topics=["user_profile", topic]
                    ),
                    user_id=user_id
                )

        # Retrieve and display the stored memories
        print(f"\nStored User Memories for {user_id}:")
        user_memories = memory.get_user_memories(user_id=user_id)
        for m in user_memories:
            print(f"\nMemory: {m.memory}")
            print(f"Topics: {m.topics}")
            print(f"Last Updated: {m.last_updated}")

        return user_id  # Return user_id for further operations

    except json.JSONDecodeError as e:
        print(f"Error decoding JSON: {e}")
        return None
    except Exception as e:
        print(f"Error storing user data: {e}")
        return None

if __name__ == "__main__":
    # Example usage with user_content variable
    user_content = '''
    {
        "Email": "r2raviteja@gmail.com",
        "password": null,
        "First name": "Raviteja",
        "Phone number": "+49 15216 442399"
    }
    '''
    # Store the data
    user_id = store_user_data(content_json)
    
    if user_id:
        # Example: Get a specific value
        phone = get_user_value(user_id, "Phone number")
        first_name = get_user_value(user_id, "Linkedin")
        print("\nRetrieved specific values:")
        print(f"Phone number: {phone}")
        print(f"First name: {first_name}")
        
        # Example: Get all values
        print("\nAll user values:")
        all_values = get_all_user_values(user_id)
        for field, value in all_values.items():
            print(f"{field}: {value}") 


Stored User Memories for r2raviteja@gmail.com:

Memory: Country: Germany
Topics: ['user_profile', 'country']
Last Updated: 2025-04-27 00:28:59.019666

Memory: password: Asdf1234
Topics: ['user_profile', 'password']
Last Updated: 2025-04-27 00:28:58.058997

Memory: First name: Raviteja
Topics: ['user_profile', 'first_name']
Last Updated: 2025-04-27 00:28:58.237432

Memory: Last name: R
Topics: ['user_profile', 'last_name']
Last Updated: 2025-04-27 00:28:58.445544

Memory: Phone number: +49 15216 442399
Topics: ['user_profile', 'phone_number']
Last Updated: 2025-04-27 00:28:58.596415

Memory: Language: English
Topics: ['user_profile', 'language']
Last Updated: 2025-04-27 00:28:58.720805

Memory: City: Munich
Topics: ['user_profile', 'city']
Last Updated: 2025-04-27 00:28:58.858029

Memory: Email: r2raviteja@gmail.com
Topics: ['user_profile', 'email']
Last Updated: 2025-04-27 00:28:57.899849

Memory: Last name: R
Topics: ['user_profile', 'last_name']
Last Updated: 2025-04-26 23:19:55.045

In [54]:
if user_id:
    # Example: Get a specific value
    #phone = get_user_value(user_id, "Phone number")
    #first_name = get_user_value(user_id, "First name")
    #print("\nRetrieved specific values:")
    #print(f"Phone number: {phone}")
    #print(f"First name: {first_name}")
    
    # Example: Get all values
    print("\nAll user values:")
    all_values = get_all_user_values(user_id="r2raviteja@gmail.com")
    for field, value in all_values.items():
        print(f"{field}: {value}") 


All user values:
Country: Germany
password: -
First name: Raviteja
Last name: R
Phone number: +49 15216 442399
Language: ['English', 'Telugu', 'German']
City: Munich
Email: r2raviteja@gmail.com
test: a


In [68]:
from datetime import datetime
from agno.agent import Agent
from agno.models.openai import OpenAIChat
from agno.tools.reasoning import ReasoningTools

# Final Premium Cover Letter Agent
cover_letter_agent = Agent(
    name="Premium Cover Letter Writer",
    role="Craft world-class, ready-to-submit cover letters",
    model=OpenAIChat(id="gpt-4.1-nano"),
    tools=[ReasoningTools(add_instructions=True)],
    instructions=[
        "GENERAL RULES:",
        "1. Write a fully polished, powerful cover letter ready for direct submission.",
        "2. 3/4 page to 1 full page (300-400 words).",
        "3. Use professional but conversational tone.",
        "4. Showcase 4+ achievements with real metrics, technologies, and impact.",
        "5. No clichés like 'hardworking' or 'team player'.",
        "6. Tailor perfectly to the job and company.",
        "7. Finish with a strong closing and a polite call to action.",
        "HEADER AND STRUCTURE RULES:",
        "1. Start with applicant's full name, email, phone, city, country.",
        "2. Today's date (example: April 27, 2025), clean, no brackets.",
        "3. Company Address: Hiring Manager, Company Name, City, Country.",
        "4. Salutation: 'Dear Hiring Manager,'",
        "5. 3-4 strong paragraphs (intro, highlights, fit, closing).",
        "6. No unnecessary information like street address if missing."
    ],
    markdown=True
)

def generate_final_cover_letter():
    today = datetime.today().strftime('%B %d, %Y')

    task = f"""
Applicant Information:
- Name: Raviteja R
- Email: r2raviteja@gmail.com
- Phone: +49 15216 442399
- Location: Munich, Germany

Date: {today}

Company Information:
- Hiring Manager
- Blacklane
- Berlin, Germany

Resume Highlights:
- 5 years of experience in data analytics and product data analysis
- Led experiments at Rapido leading to 5% improvement in retention through ScratchCard Experiment
- Created predictive dashboards in Power BI for sales steering across 119 countries at Allianz
- Developed AllianzGPT (internal ChatGPT) using LangChain and OpenAI for company-wide knowledge access
- Experience with SQL, Python, Metabase, Power BI, and A/B testing
- Certified Data Scientist

Job Applied: Senior Data Analyst at Blacklane

Instructions:
- Write a full cover letter using above details.
- Highlight 4+ achievements with metrics (example: 'improved retention by 5% across 1 lakh users').
- Connect achievements directly to skills needed at Blacklane.
- Assume Blacklane values innovation, data-driven insights, customer-first solutions.
- Keep tone professional but slightly personal, enthusiastic about opportunity.
- End with a strong call to action (ex: looking forward to discussing how I can contribute).

Now write the full cover letter following all structure and formatting rules.
"""

    return cover_letter_agent.run(task)

if __name__ == "__main__":
    cover_letter = generate_final_cover_letter()
    print(cover_letter)


RunResponse(content='Raviteja R  \nr2raviteja@gmail.com  \n+49 15216 442399  \nMunich, Germany  \n\nApril 27, 2025  \n\nHiring Manager  \nBlacklane  \nBerlin, Germany  \n\nDear Hiring Manager,  \n\nI am excited to apply for the Senior Data Analyst position at Blacklane. With over five years of experience in data analytics and product data analysis, I have developed expertise in transforming complex data into actionable insights that drive customer-centric solutions and business growth. My background in leading innovative experiments, designing predictive dashboards, and building internal AI tools aligns well with Blacklane’s commitment to leveraging data for exceptional mobility experiences.  \n\nAt Rapido, I led experiments that resulted in a 5% boost in user retention through a ScratchCard experiment, exemplifying my ability to harness A/B testing and data-driven strategies for measurable impact. During my tenure at Allianz, I crafted comprehensive Power BI dashboards that successful

In [78]:
with open("test.txt", 'w', encoding='utf-8') as f:
        f.write(cover_letter.content)

In [70]:
resume_text = cover_letter.content
resume_text

'Raviteja R  \nr2raviteja@gmail.com  \n+49 15216 442399  \nMunich, Germany  \n\nApril 27, 2025  \n\nHiring Manager  \nBlacklane  \nBerlin, Germany  \n\nDear Hiring Manager,  \n\nI am excited to apply for the Senior Data Analyst position at Blacklane. With over five years of experience in data analytics and product data analysis, I have developed expertise in transforming complex data into actionable insights that drive customer-centric solutions and business growth. My background in leading innovative experiments, designing predictive dashboards, and building internal AI tools aligns well with Blacklane’s commitment to leveraging data for exceptional mobility experiences.  \n\nAt Rapido, I led experiments that resulted in a 5% boost in user retention through a ScratchCard experiment, exemplifying my ability to harness A/B testing and data-driven strategies for measurable impact. During my tenure at Allianz, I crafted comprehensive Power BI dashboards that successfully steered sales ini

In [71]:
from agno.agent import Agent
from agno.models.openai import OpenAIChat

# Cover Letter Reviewer Agent
cover_sanitizer = Agent(
    name="Nuclear Cover Letter Sanitizer",
    role="Remove ANY remaining artifacts with zero content changes",
    model=OpenAIChat(id="gpt-4.1-nano"),
    instructions=[
        "ONLY PERFORM THESE ACTIONS:",
        "1. Remove if exists:",
        "   - Any text before name line",
        "   - Markdown symbols (** --- ==)",
        "   - Empty square brackets []",
        "2. NEVER:",
        "   - Modify any names/dates/numbers",
        "   - Change formatting or spacing",
        "   - Add/remove content",
        "3. FAILSAFE: If uncertain → return original"
    ],
    markdown=False
)


def generate_cover_letter(resume_text: str, job_description: str, 
                        applicant_info: dict, company_info: dict) -> str:
    """
    Generates perfectly formatted, submission-ready cover letters
    """
    task = f"""Generate a FINAL DRAFT cover letter ready for immediate submission with these strict requirements:

1. FORMATTING RULES:
   - Start directly with name (no titles, no stars)
   - Contact info: [Name] [Newline] [Address] [Newline] [Email] | [Phone]
   - One empty line before date
   - Current date: {datetime.now().strftime("%B %d, %Y")}
   - One empty line before company info
   - Salutation: "Dear Hiring Team,"

2. CONTENT RULES:
   - Use ONLY information from provided resume and job description
   - Never use placeholders like [City] or [Company Address]
   - Never add "Here is..." or "This letter..." intros
   - Never include section headers or markdown

3. STRUCTURE:
   {applicant_info['First name']} {applicant_info['Last name']}
   {applicant_info['City']}, {applicant_info['Country']}
   {applicant_info['Email']} | {applicant_info['Phone number']}

   {datetime.now().strftime("%B %d, %Y")}

   Hiring Team
   {company_info['name']}

   Dear Hiring Team,

   [Opening: 1-2 sentences connecting to role]
   [Body 1: Top 2 achievements with metrics]
   [Body 2: Technical skills matching job]
   [Closing: Call to action]

   Best regards,
   {applicant_info['First name']} {applicant_info['Last name']}

4. INPUT DATA:
   Resume: {resume_text}
   Job Description: {job_description}
   Company: {company_info['name']} ({company_info.get('industry', '')})
   """

    return cover_sanitizer.run(task)

# Modified generate_cover_letter function
draft = generate_cover_letter(resume_text, JD, applicant, company)
draft

RunResponse(content='Raviteja R  \nMunich, Germany  \nr2raviteja@gmail.com | +49 15216 442399\n\nApril 27, 2025\n\nHiring Manager  \nBlacklane  \nBerlin, Germany\n\nDear Hiring Manager,\n\nI am excited to apply for the Senior Data Analyst position at Blacklane. With over five years of experience in data analytics and product data analysis, I have developed expertise in transforming complex data into actionable insights that drive customer-centric solutions and business growth. My background in leading innovative experiments, designing predictive dashboards, and building internal AI tools aligns well with Blacklane’s commitment to leveraging data for exceptional mobility experiences.\n\nAt Rapido, I led experiments that resulted in a 5% boost in user retention through a ScratchCard experiment, exemplifying my ability to harness A/B testing and data-driven strategies for measurable impact. During my tenure at Allianz, I crafted comprehensive Power BI dashboards that successfully steered 

In [66]:
with open("draft.txt", 'w', encoding='utf-8') as f:
        f.write(draft.content)

In [72]:
import textwrap
import re

def wrap_paragraphs(text, width=80):
    # Split the text into paragraphs based on double newlines or three or more spaces
    paragraphs = re.split(r'\n\n|\s{3,}', text)
    # Wrap each paragraph individually and join them back with double newlines
    wrapped_paragraphs = '\n\n'.join([textwrap.fill(paragraph, width=width) for paragraph in paragraphs])
    return wrapped_paragraphs

with open("draft.txt", 'w', encoding='utf-8') as f:
    wrapped_text = wrap_paragraphs(draft.content, 120)
    f.write(wrapped_text)


In [35]:
from docx import Document

def save_to_word(content: str, filename: str = "cover_letter.docx"):
    """Save content to a Word document"""
    doc = Document()
    for paragraph in content.split('\n\n'):
        if paragraph.strip():
            doc.add_paragraph(paragraph.strip())
    doc.save(filename)
save_to_word(draft.content)

In [13]:
from agno.agent import Agent
from agno.models.openai import OpenAIChat
from agno.tools.reasoning import ReasoningTools

# Cover Letter Writer Agent
cover_letter_agent = Agent(
    name="Cover Letter Writer",
    role="Write professional and compelling cover letters",
    model=OpenAIChat(id="gpt-4.1-nano"),
    tools=[ReasoningTools(add_instructions=True)],
    instructions=[
        "Write professional and compelling cover letters",
        "Use formal business letter format",
        "Create engaging opening paragraphs",
        "Match qualifications with job requirements",
        "Include specific examples and achievements",
        "Maintain professional tone",
        "Keep content concise and impactful",
        "End with clear call to action"
    ],
    markdown=True
)

def generate_cover_letter(resume_text: str, job_description: str, applicant_info: dict, company_info: dict) -> str:
    """
    Generate a cover letter using the agent
    
    Args:
        resume_text: Full text of the resume/CV
        job_description: Full job description
        applicant_info: Dictionary with applicant's contact details
        company_info: Dictionary with company and position details
    
    Returns:
        str: The final cover letter
    """
    task = f"""
    Generate a professional cover letter with the following inputs:

    RESUME:
    {resume_text}

    JOB DESCRIPTION:
    {job_description}

    APPLICANT INFORMATION:
    {applicant_info}

    COMPANY INFORMATION:
    {company_info}
    """
    
    return cover_letter_agent.print_response(task, stream=True)

if __name__ == "__main__":
    # Example usage
    resume = """[Your resume text here]"""
    job_desc = """[Job description text here]"""
    applicant = {
        "name": "John Doe",
        "email": "john@example.com",
        "phone": "(555) 123-4567",
        "linkedin": "linkedin.com/in/johndoe"
    }
    company = {
        "name": "Tech Corp",
        "hiring_manager": "Jane Smith",
        "position": "Senior Software Engineer"
    }
    
    # Generate and print cover letter
    cover_letter = generate_cover_letter(CV, JD, applicant, company)
    print(cover_letter) 

Output()

None
